In [1]:
import pandas as pd
import numpy as np
import sidetable
import pymongo

import folium
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append("../")
import src.creacion_columnas as cc

### Importamos CSV

Nos descarmos el CSV que hemos creado, enriqueciendo el CSV inicial con Selenium:

In [3]:
df3 = pd.read_csv ("../Data/03. Datos_mergeados_.csv")
df3.head(2)

,Unnamed: 0,Price,Currency,Brokered_by,AD_Situation,Rooms,Bath,Building space,Measruing unit,Address,...,State,Índice_poder_adqui,Índice_seguridad,Índice_sanidad,Índice_clima,Índice_costo_vida,Relación_precio_ingresos_propiedades,Índice_costo_desplazamiento,Índice_contaminación,Índice_calidad_vida
0,0,289900,USD,Greene Realty Group LLC,New - 7 hours ago,3,2.0,1944,sqft,"5142 Puget Rd NE, Olympia, WA 98516",...,WA 98516,"110,78","70,61","71,99","83,44","70,06","3,37","19,11","16,01","205,63"
1,1,146999,USD,Adair Homes,Old,3,2.0,1340,sqft,"South Puget Sound 2303 93rd Ave SW, Olympia, W...",...,WA 98512,"110,78","70,61","71,99","83,44","70,06","3,37","19,11","16,01","205,63"


### Manipulación de columnas

Vamos a reescribir la columna "AD_Situation" para que únicamente muestre dos opciones: New, old. 

Vamos a reescribir la columna "State" para que únicamente muestra la primera parte de los string, es decir: WA, OR o CA.

Para ello, hemos creado una función "quitar_horas" donde haremos un split por espacio y nos quedaremos con la primera parte del string!

In [4]:
"""def quitar_horas(x): 
    try:
        return x.split()[0]
    except: 
        return x"""

'def quitar_horas(x): \n    try:\n        return x.split()[0]\n    except: \n        return x'

In [5]:
df3["AD_Situation"] = df3["AD_Situation"].apply(cc.quitar_horas)
df3["State"] = df3["State"].apply(cc.quitar_horas)
df3.head(1)

,Unnamed: 0,Price,Currency,Brokered_by,AD_Situation,Rooms,Bath,Building space,Measruing unit,Address,...,State,Índice_poder_adqui,Índice_seguridad,Índice_sanidad,Índice_clima,Índice_costo_vida,Relación_precio_ingresos_propiedades,Índice_costo_desplazamiento,Índice_contaminación,Índice_calidad_vida
0,0,289900,USD,Greene Realty Group LLC,New,3,2.0,1944,sqft,"5142 Puget Rd NE, Olympia, WA 98516",...,WA,"110,78","70,61","71,99","83,44","70,06","3,37","19,11","16,01","205,63"


De nuevo vamos a reescriir la columna "State", pero para que este vez muestre los 3 Estados en su nomnbre completo: California, Oregón, Washington: 

Para ello hemos creado la función "cambiar"!

In [6]:
""""def cambiar(x):
    if x == "WA":
        return x.replace("WA", "Washington")

    elif  x == "OR":
        return x.replace("OR", "Oregón")

    else:
        return x.replace("CA", "California")"""

'"def cambiar(x):\n    if x == "WA":\n        return x.replace("WA", "Washington")\n\n    elif  x == "OR":\n        return x.replace("OR", "Oregón")\n\n    else:\n        return x.replace("CA", "California")'

In [7]:
df3["State"] = df3.apply(lambda datos: cc.cambiar(datos["State"]),axis = 1)

Vamos a crear una nueva columna "Measuring unit" y vamos a indicar como unidad de medida "m2".

Vamos a reescribir la columna "Bulding space" para que la información se muestra en m2 en vez de sqft. Para ello lo vamos a dividir por 10.764.

Vamos a crear una columna nueva "Price/m2", donde vamos dividir la columna "Price" por la columna "Building space". 

In [8]:
df3['Measuring unit']= df3['Measruing unit'].replace("sqft","m2")
df3['Building space']= round(df3['Building space']/10.764, 0)
df3['Price/m2']= round((df3['Price'])/(df3['Building space']), 0)
df3.head(1)

,Unnamed: 0,Price,Currency,Brokered_by,AD_Situation,Rooms,Bath,Building space,Measruing unit,Address,...,Índice_seguridad,Índice_sanidad,Índice_clima,Índice_costo_vida,Relación_precio_ingresos_propiedades,Índice_costo_desplazamiento,Índice_contaminación,Índice_calidad_vida,Measuring unit,Price/m2
0,0,289900,USD,Greene Realty Group LLC,New,3,2.0,181.0,sqft,"5142 Puget Rd NE, Olympia, WA 98516",...,"70,61","71,99","83,44","70,06","3,37","19,11","16,01","205,63",m2,1602.0


Vamos a crear tres columnas nuevas, "Direction", "Ciudad1", "Postal_code" a partir de la columna "Address", donde vamos a hacer un split por ",", y nos vamos a quedar con las 3 primeras palabras del string:

In [9]:
df3[["Direction", "Ciudad1", "Postal_code"]]  = df3["Address"].str.split(",",  expand=True).iloc[:, :3]

Vamos a reescribir la columna "Direction" para que nos de la dirección y la ciudad en la misma columna. Para ello, vamos a sumar los strings de las columnas "Direction" y "Ciudad1":

In [10]:
df3["Direction"] = df3["Direction"] + "," + df3["Ciudad1"]
df3.head(1)

,Unnamed: 0,Price,Currency,Brokered_by,AD_Situation,Rooms,Bath,Building space,Measruing unit,Address,...,Índice_costo_vida,Relación_precio_ingresos_propiedades,Índice_costo_desplazamiento,Índice_contaminación,Índice_calidad_vida,Measuring unit,Price/m2,Direction,Ciudad1,Postal_code
0,0,289900,USD,Greene Realty Group LLC,New,3,2.0,181.0,sqft,"5142 Puget Rd NE, Olympia, WA 98516",...,"70,06","3,37","19,11","16,01","205,63",m2,1602.0,"5142 Puget Rd NE, Olympia",Olympia,WA 98516


Vamos a reescribir la columna "Postal_code" para que nos de únicamente el código postal: 

Para ello, utilizando regex, vamos a quedarnos únicamente con los valores numéricos del string

In [11]:
df3["Postal_code"] = df3.Postal_code.str.extract("(\d+)", expand=True)
df3.head(1)

,Unnamed: 0,Price,Currency,Brokered_by,AD_Situation,Rooms,Bath,Building space,Measruing unit,Address,...,Índice_costo_vida,Relación_precio_ingresos_propiedades,Índice_costo_desplazamiento,Índice_contaminación,Índice_calidad_vida,Measuring unit,Price/m2,Direction,Ciudad1,Postal_code
0,0,289900,USD,Greene Realty Group LLC,New,3,2.0,181.0,sqft,"5142 Puget Rd NE, Olympia, WA 98516",...,"70,06","3,37","19,11","16,01","205,63",m2,1602.0,"5142 Puget Rd NE, Olympia",Olympia,98516


Vamos a eliminar las columna que no nos interesan por estar la información "duplicada"

In [12]:
eliminar_columns = ['Address', 'Unnamed: 0', 'Ciudad1', 'Measruing unit']


In [13]:
df3_limpio = df3.drop(eliminar_columns, axis = 1)
df3_limpio.head(1)

,Price,Currency,Brokered_by,AD_Situation,Rooms,Bath,Building space,City,State,Índice_poder_adqui,...,Índice_clima,Índice_costo_vida,Relación_precio_ingresos_propiedades,Índice_costo_desplazamiento,Índice_contaminación,Índice_calidad_vida,Measuring unit,Price/m2,Direction,Postal_code
0,289900,USD,Greene Realty Group LLC,New,3,2.0,181.0,Olympia,Washington,"110,78",...,"83,44","70,06","3,37","19,11","16,01","205,63",m2,1602.0,"5142 Puget Rd NE, Olympia",98516


### Estandarización del tipado de las columnas

Vamos a ordenar las columnas de nuestro DataFrame para que tenga algo más de sentido:

In [14]:
df3_limpio = df3_limpio.reindex(columns=['Price', 'Currency', 'AD_Situation', 'Rooms', 'Bath', 'Building space', 'Measuring unit', 'Price/m2', 'Brokered_by', 'Direction', 'City', 'State', 'Postal_code',
       'Índice_poder_adqui', 'Índice_seguridad', 'Índice_sanidad',
       'Índice_clima', 'Índice_costo_vida',
       'Relación_precio_ingresos_propiedades', 'Índice_costo_desplazamiento',
       'Índice_contaminación', 'Índice_calidad_vida'])
df3_limpio.head(1)

,Price,Currency,AD_Situation,Rooms,Bath,Building space,Measuring unit,Price/m2,Brokered_by,Direction,...,Postal_code,Índice_poder_adqui,Índice_seguridad,Índice_sanidad,Índice_clima,Índice_costo_vida,Relación_precio_ingresos_propiedades,Índice_costo_desplazamiento,Índice_contaminación,Índice_calidad_vida
0,289900,USD,New,3,2.0,181.0,m2,1602.0,Greene Realty Group LLC,"5142 Puget Rd NE, Olympia",...,98516,"110,78","70,61","71,99","83,44","70,06","3,37","19,11","16,01","205,63"


A continuación, vamos a homogeneizar las columnas: pasar los nombres de las columnas a minusculas y reemplazar los espacios por "_":

In [15]:
diccionario = {}
for col in df3_limpio.columns:
    diccionario[col] = col.lower().strip().replace(" ", "_")

In [16]:
nuevas_columnas = {col : col.lower().strip().replace(" ", "_") for col in df3_limpio.columns}
nuevas_columnas

{'Price': 'price',
 'Currency': 'currency',
 'AD_Situation': 'ad_situation',
 'Rooms': 'rooms',
 'Bath': 'bath',
 'Building space': 'building_space',
 'Measuring unit': 'measuring_unit',
 'Price/m2': 'price/m2',
 'Brokered_by': 'brokered_by',
 'Direction': 'direction',
 'City': 'city',
 'State': 'state',
 'Postal_code': 'postal_code',
 'Índice_poder_adqui': 'índice_poder_adqui',
 'Índice_seguridad': 'índice_seguridad',
 'Índice_sanidad': 'índice_sanidad',
 'Índice_clima': 'índice_clima',
 'Índice_costo_vida': 'índice_costo_vida',
 'Relación_precio_ingresos_propiedades': 'relación_precio_ingresos_propiedades',
 'Índice_costo_desplazamiento': 'índice_costo_desplazamiento',
 'Índice_contaminación': 'índice_contaminación',
 'Índice_calidad_vida': 'índice_calidad_vida'}

In [17]:
df3_limpio.rename(columns = nuevas_columnas, inplace =True)
df3_limpio.head(1)

,price,currency,ad_situation,rooms,bath,building_space,measuring_unit,price/m2,brokered_by,direction,...,postal_code,índice_poder_adqui,índice_seguridad,índice_sanidad,índice_clima,índice_costo_vida,relación_precio_ingresos_propiedades,índice_costo_desplazamiento,índice_contaminación,índice_calidad_vida
0,289900,USD,New,3,2.0,181.0,m2,1602.0,Greene Realty Group LLC,"5142 Puget Rd NE, Olympia",...,98516,"110,78","70,61","71,99","83,44","70,06","3,37","19,11","16,01","205,63"


### Tipo de Datos

Problema: La columna "baños" está en formato "float". Debemos convertirlas a integer. Las columans "índice_poder_adqui', 'índice_seguridad', 'índice_sanidad', 'índice_clima', 'índice_costo_vida', 'relación_precio_ingresos_propiedades', 'índice_costo_desplazamiento', 'índice_contaminación', 'índice_calidad_vida' están en formato "string"... Tendremos que hacer algo con ellas más adelante. Debemos convertirlas a integer:

In [18]:
df3_limpio.dtypes

price                                     int64
currency                                 object
ad_situation                             object
rooms                                     int64
bath                                    float64
building_space                          float64
measuring_unit                           object
price/m2                                float64
brokered_by                              object
direction                                object
city                                     object
state                                    object
postal_code                              object
índice_poder_adqui                       object
índice_seguridad                         object
índice_sanidad                           object
índice_clima                             object
índice_costo_vida                        object
relación_precio_ingresos_propiedades     object
índice_costo_desplazamiento              object
índice_contaminación                    

In [19]:
df3_limpio["bath"] = df3_limpio["bath"].round(1).astype(int)
df3_limpio.head(2)

,price,currency,ad_situation,rooms,bath,building_space,measuring_unit,price/m2,brokered_by,direction,...,postal_code,índice_poder_adqui,índice_seguridad,índice_sanidad,índice_clima,índice_costo_vida,relación_precio_ingresos_propiedades,índice_costo_desplazamiento,índice_contaminación,índice_calidad_vida
0,289900,USD,New,3,2,181.0,m2,1602.0,Greene Realty Group LLC,"5142 Puget Rd NE, Olympia",...,98516,"110,78","70,61","71,99","83,44","70,06","3,37","19,11","16,01","205,63"
1,146999,USD,Old,3,2,124.0,m2,1185.0,Adair Homes,"South Puget Sound 2303 93rd Ave SW, Olympia",...,98512,"110,78","70,61","71,99","83,44","70,06","3,37","19,11","16,01","205,63"


In [20]:
x = ['índice_poder_adqui', 'índice_seguridad',
       'índice_sanidad', 'índice_clima', 'índice_costo_vida',
       'relación_precio_ingresos_propiedades', 'índice_costo_desplazamiento',
       'índice_contaminación', 'índice_calidad_vida']

def puntos(df,x):
    for i in x:
        
        df3_limpio[i] = df3_limpio [i].str.replace(",","")
        df3_limpio[i] = pd.to_numeric(df[i], errors = "coerce").astype("Int64")


In [21]:
puntos(df3_limpio, x)

In [22]:
df3_limpio.dtypes

price                                     int64
currency                                 object
ad_situation                             object
rooms                                     int64
bath                                      int64
building_space                          float64
measuring_unit                           object
price/m2                                float64
brokered_by                              object
direction                                object
city                                     object
state                                    object
postal_code                              object
índice_poder_adqui                        Int64
índice_seguridad                          Int64
índice_sanidad                            Int64
índice_clima                              Int64
índice_costo_vida                         Int64
relación_precio_ingresos_propiedades      Int64
índice_costo_desplazamiento               Int64
índice_contaminación                    

### Duplicados

Problema: Duplicado. En el análisis exploratorio identificamos 2 duplicados:

In [23]:
df3_limpio.duplicated().sum()

2

In [24]:
df3_limpio[df3_limpio.duplicated() == True]

,price,currency,ad_situation,rooms,bath,building_space,measuring_unit,price/m2,brokered_by,direction,...,postal_code,índice_poder_adqui,índice_seguridad,índice_sanidad,índice_clima,índice_costo_vida,relación_precio_ingresos_propiedades,índice_costo_desplazamiento,índice_contaminación,índice_calidad_vida
461,484000,USD,New,3,2,250.0,m2,1936.0,Pacific Lifestyle Homes,"Adam's Glen 16701 NE 88th St, Vancouver",...,98682,9044,6257,7477,9115,7331,1178,3690,2501,17353
503,394000,USD,New,3,2,135.0,m2,2919.0,Pacific Lifestyle Homes,"Parkers Abby 11203 NE 133rd Pl., Vancouver",...,98682,9044,6257,7477,9115,7331,1178,3690,2501,17353


Una vez identificamos las filas duplicadas, procedemos a su eliminación:

In [25]:
df3_limpio.drop_duplicates(inplace = True)

In [26]:
df3_limpio.duplicated().sum()

0

### Valores Nulos

In [27]:
df3_limpio.isnull().sum()

price                                   0
currency                                0
ad_situation                            0
rooms                                   0
bath                                    0
building_space                          0
measuring_unit                          0
price/m2                                0
brokered_by                             0
direction                               0
city                                    0
state                                   0
postal_code                             2
índice_poder_adqui                      0
índice_seguridad                        0
índice_sanidad                          0
índice_clima                            0
índice_costo_vida                       0
relación_precio_ingresos_propiedades    0
índice_costo_desplazamiento             0
índice_contaminación                    0
índice_calidad_vida                     0
dtype: int64

In [28]:
df3_limpio["postal_code"] = df3_limpio["postal_code"].fillna(0)

In [29]:
df3_limpio.isnull().sum()

price                                   0
currency                                0
ad_situation                            0
rooms                                   0
bath                                    0
building_space                          0
measuring_unit                          0
price/m2                                0
brokered_by                             0
direction                               0
city                                    0
state                                   0
postal_code                             0
índice_poder_adqui                      0
índice_seguridad                        0
índice_sanidad                          0
índice_clima                            0
índice_costo_vida                       0
relación_precio_ingresos_propiedades    0
índice_costo_desplazamiento             0
índice_contaminación                    0
índice_calidad_vida                     0
dtype: int64

In [30]:
df3_limpio["postal_code"].dtypes

dtype('O')

In [31]:
df3_limpio["postal_code"] = df3_limpio["postal_code"].astype(int)

In [32]:
df3_limpio["postal_code"].dtypes

dtype('int64')

### Principales estadísticos

In [33]:
df3_limpio.describe().T

,count,mean,std,min,25%,50%,75%,max
price,683.0,592977.174231,423870.700956,27900.0,339700.0,449900.0,700000.0,3580000.0
rooms,683.0,3.207906,1.065644,1.0,3.0,3.0,4.0,8.0
bath,683.0,2.011713,0.871432,1.0,2.0,2.0,2.0,8.0
building_space,683.0,180.346999,79.253526,42.0,122.5,168.0,225.5,585.0
price/m2,683.0,3418.598829,1812.137456,297.0,2252.5,2914.0,4222.0,15049.0
postal_code,683.0,97478.200586,5504.174697,0.0,97237.5,98178.0,98506.0,98686.0
índice_poder_adqui,683.0,11487.631040,2479.533594,9044.0,9240.0,10626.0,13635.0,18897.0
índice_seguridad,683.0,5665.431918,1150.258067,4290.0,4843.0,4932.0,7061.0,8514.0
índice_sanidad,683.0,7405.383602,216.472085,7199.0,7318.0,7319.0,7407.0,8685.0
índice_clima,683.0,8952.026354,438.735014,6188.0,8956.0,9088.0,9173.0,9682.0


In [34]:
df3_limpio.describe(include = "object").T

,count,unique,top,freq
currency,683,1,USD,683
ad_situation,683,2,New,625
measuring_unit,683,1,m2,683
brokered_by,683,283,Adair Homes,29
direction,683,633,"South Puget Sound 2303 93rd Ave SW, Olympia",29
city,683,12,Portland,137
state,683,3,Washington,504


### Geopy

In [35]:
from geopy.geocoders import Nominatim

In [36]:
locator = Nominatim(user_agent= "myGeocoder")

In [37]:
"""location = locator.geocode ("98516")
print (location)"""

'location = locator.geocode ("98516")\nprint (location)'

In [38]:
"""print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))"""

'print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))'

In [39]:
"""location.latitude"""

'location.latitude'

In [40]:
def latitud_longitud (x):
    locator = Nominatim(user_agent= "myGeocoder")
    try:
        
        location = locator.geocode (x)
        return location.latitude, location.longitude
        
    except:
        return None

In [41]:
latitud_longitud (98516)

(47.1342656625, -122.73493423336308)

In [42]:
"""df3_limpio[["latitud", "longitud"]] = df3_limpio.apply(lambda datos: latitud_longitud(
                                    datos["postal_code"]), axis = 1)"""

'df3_limpio[["latitud", "longitud"]] = df3_limpio.apply(lambda datos: latitud_longitud(\n                                    datos["postal_code"]), axis = 1)'

In [43]:
"""df3_limpio[["latitud", "longitud"]] = df3_limpio["postal_code"].apply(latitud_longitud).split(",")

'df3_limpio[["latitud", "longitud"]] = df3_limpio["postal_code"].apply(latitud_longitud)'

In [44]:
def latitud_v2 (x):
    locator = Nominatim(user_agent= "myGeocoder")
    try:
        
        location = locator.geocode (x)
        return location.latitude, location.longitud
        
    except:
        return None

def longitud_v2 (x):
    locator = Nominatim(user_agent= "myGeocoder")
    try:
        
        location = locator.geocode (x)
        return location.longitude
        
    except:
        return None

In [45]:
def hola(x):
    return "Hola", "adios"

In [46]:
x, y = hola()

TypeError: hola() missing 1 required positional argument: 'x'

In [ ]:
xñkhj

In [ ]:
y

In [ ]:
df3_limpio[["laitud", "longtud"]] = df3_limpio["postal_code"].apply(hola)

In [ ]:
df3_limpio["latidud_"] = df3_limpio.apply(lambda datos: latitud_v2(
                                    datos["postal_code"]), axis = 1)

In [ ]:
df3_limpio["longitud_"] = df3_limpio.apply(lambda datos: longitud_v2(
                                    datos["postal_code"]), axis = 1)

### Exportación de CSV limpio

Exportamos el DataFrame limpio a un CSV:

In [ ]:
df3_limpio.to_csv("04. Datos_limpitos_.csv")

In [ ]:
"""dd = cc.latitud_v2("98516")"""

In [ ]:
dd

In [ ]:
mapa=folium.Map(location=[47.134266, -122.734934],  # la posicion
                tiles='openstreetmap',            # tipo de mapa, theme
                zoom_start=10)                    # zoom inicial

mapa

In [ ]:
"""for index, row in df.iterrows(): # método para iterar por los dataframes

cities = ['Olympia', 'Seattle', 'Des Moines', 'Bellevue', 'Redmond', 'Tacoma', 'Vancouver', 'Long Beach', 'Pasadena', 'Anaheim', 'Irvine', 'Portland']
    for i in cities:
        try:
                row['city'] == (i) :

        return folium.Marker([row["latitud_"], row['longitud_']],
                              radius=10, 
                              tooltip = row[['is_happy_customer', 'city']],
                              icon=folium.Icon(icon = "cloud", color = "red")).add_to(mapa)
        except:
            None"""

In [ ]:
df3_limpio

In [ ]:
for index, row in df3_limpio.iterrows():
    try:
        if row['city'] == "Olympia":

            folium.Marker([row["latitud_"], row['longitud_']],
                        radius=10,
                        icon=folium.Icon(icon = "cloud", color = "red")).add_to(mapa)
        elif row['city']  == "Seattle":
            folium.Marker([row["latitud_"], row['longitud_']],
                        radius=10, 
                        icon=folium.Icon(icon = "cloud", color = "orange")).add_to(mapa)

        elif row['city']  == "Des Moines":
            folium.Marker([row["latitud_"], row['longitud_']],
                        radius=10,
                        icon=folium.Icon(icon = "cloud", color = "black")).add_to(mapa)   
        elif row['city']  == "Bellevue":
            folium.Marker([row["latitud_"], row['longitud_']],
                        radius=10,
                        icon=folium.Icon(icon = "cloud", color = "yellow")).add_to(mapa)

        elif row['city']  == "Redmond":
                folium.Marker([row["latitud_"], row['longitud_']],
                        radius=10,
                        icon=folium.Icon(icon = "cloud", color = "green")).add_to(mapa)  
        elif row['city']  == "Tacoma":
            folium.Marker([row["latitud_"], row['longitud_']],
                        radius=10,
                        icon=folium.Icon(icon = "cloud", color = "purple")).add_to(mapa)

        elif row['city']  == "Vancouver":
            folium.Marker([row["latitud_"], row['longitud_']],
                        radius=10,
                        icon=folium.Icon(icon = "cloud", color = "blue")).add_to(mapa) 

        elif row['city']  == "Long Beach":
            folium.Marker([row["latitud_"], row['longitud_']],
                        radius=10,
                        icon=folium.Icon(icon = "cloud", color = "grey")).add_to(mapa)

        elif row['city']  == "Pasadena":
             folium.Marker([row["latitud_"], row['longitud_']],
                        radius=10, 
                        icon=folium.Icon(icon = "cloud", color = "pink")).add_to(mapa)  
        elif row['city']  == "Anaheim":
            folium.Marker([row["latitud_"], row['longitud_']],
                        radius=10, 
                        icon=folium.Icon(icon = "cloud", color = "violet")).add_to(mapa)

        elif row['city']  == "Irvine":
                folium.Marker([row["latitud_"], row['longitud_']],
                        radius=10, 
                        icon=folium.Icon(icon = "cloud", color = "plum")).add_to(mapa)
        else:
            folium.Marker([row["latitud_"], row['longitud_']],
                        radius=10, 
                        icon=folium.Icon(icon = "cloud", color = "brown")).add_to(mapa)
    except:
        pass
            


In [ ]:
mapa

In [ ]:
"""for index, row in df3_limpio.iterrows(): # método para iterar por los dataframes
    if row['city'] == "Olympia":
    
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10,
                    icon=folium.Icon(icon = "cloud", color = "red")).add_to(mapa)
    elif row['city']  == "Seattle":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    icon=folium.Icon(icon = "cloud", color = "orange")).add_to(mapa)
        
    elif row['city']  == "Des Moines":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10,
                    icon=folium.Icon(icon = "cloud", color = "black")).add_to(mapa)   
    elif row['city']  == "Bellevue":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10,
                    icon=folium.Icon(icon = "cloud", color = "yellow")).add_to(mapa)
        
    elif row['city']  == "Redmond":
            folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10,
                    icon=folium.Icon(icon = "cloud", color = "green")).add_to(mapa)  
    elif row['city']  == "Tacoma":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10,
                    icon=folium.Icon(icon = "cloud", color = "purple")).add_to(mapa)
        
    elif row['city']  == "Vancouver":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10,
                    icon=folium.Icon(icon = "cloud", color = "blue")).add_to(mapa) 
    
    elif row['city']  == "Long Beach":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10,
                    icon=folium.Icon(icon = "cloud", color = "grey")).add_to(mapa)
        
    elif row['city']  == "Pasadena":
         folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    icon=folium.Icon(icon = "cloud", color = "pink")).add_to(mapa)  
    elif row['city']  == "Anaheim":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    icon=folium.Icon(icon = "cloud", color = "violet")).add_to(mapa)
        
    elif row['city']  == "Irvine":
            folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    icon=folium.Icon(icon = "cloud", color = "plum")).add_to(mapa)
    else:
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    icon=folium.Icon(icon = "cloud", color = "brown")).add_to(mapa)"""

In [ ]:
df_line = pd.DataFrame(df3_limpio.groupby([df3_limpio['city']])["price"].mean())

In [ ]:
df_line.head(3)

In [ ]:
ax = plt.axes() # para poder cambiar el estilo de los ejes

ax.set_facecolor("white") # poner el fondo en blanco

ax.spines['bottom'].set_color('black') # color del eje x
ax.spines['top'].set_color('white') # color del eje de arriba
ax.spines['right'].set_color('white') # color del eje de la derecha
ax.spines['left'].set_color('black') # color del eje y

plt.plot(df_line['city'], # eje x
         df_line["price"], # eje y
         color = "black", # para cambiar el color de la línea
         linewidth = 2, # para cambiar el grosor de la línea
         marker = "o") # para poner los marcadores en nuestra línea

plt.xlabel("ciudad") # para poner etiqueta en el eje x
plt.ylabel("precio") # para poner etiqueta en el eje y


plt.show();

In [ ]:
sns.scatterplot(data=df3_limpio, x='índice_poder_adqui', y='price/m2',
                s = 100); # tamaño de los marcadores

### Relación Espacio de vivienda y precio

In [ ]:
sns.scatterplot(data=df3_limpio, x='building_space', y='price',
                  hue = "city",
                s = 100); # tamaño de los marcadores

In [ ]:
df_pie = pd.DataFrame(df3_limpio.groupby(['city'])
                      ["price/m2"].mean()).reset_index()

df_pie.head()

In [ ]:
df_pie_1 = pd.DataFrame(df3_limpio.groupby(['city'])
                      ["índice_poder_adqui"].mean()).reset_index()

df_pie_1.head()

In [ ]:
df_pie_1 = pd.DataFrame(df3_limpio.groupby(['city'])
                      ["índice_seguridad"].mean()).reset_index()

df_pie_1.head()

In [ ]:
sns.scatterplot(data=df3_limpio, x='state', y='índice_contaminación',
                s = 100); # tamaño de los marcadores

In [ ]:
sns.scatterplot(data=df3_limpio, x='índice_contaminación', y='índice_calidad_vida',
                s = 100); # tamaño de los marcadores

In [ ]:
sns.scatterplot(data=df3_limpio, x='relación_precio_ingresos_propiedades', y='price',
                s = 100); # tamaño de los marcadores

In [ ]:
sns.scatterplot(data=df3_limpio, x='state', y='índice_calidad_vida',
                s = 100); # tamaño de los marcadores

In [ ]:
sns.scatterplot(data=df3_limpio, x='sanidad', y='índice_contaminación',
                s = 100); # tamaño de los marcadores

In [ ]:
def explorar(df, ciudad):
    city = df[df["city"] == f"{ciudad}"]
    print("los principales estadisticos son: ")
    print(city['price/m2'].max())
    print("-----------")
    

In [ ]:
def explorar_min(df, ciudad):
    city = df[df["city"] == f"{ciudad}"]
    print("los principales estadisticos son: ")
    print(city['price/m2'].min())
    print("-----------")

In [ ]:
def explorar_v2(df, ciudad):
    city = df[df["city"] == f"{ciudad}"]
    print("los principales estadisticos son: ")
    print df3_limipio().T

In [ ]:
explorar(df3_limpio, "Irvine")

In [ ]:
explorar_min(df3_limpio, "Irvine")

In [ ]:
df3_limpio.groupby(["state", 'city'])["price/m2"].mean()

In [ ]:
df3_limpio.groupby(["state", 'city'])["price/m2"].max()

In [ ]:
df3_limpio.groupby(["state", 'city'])["price/m2"].min()

In [ ]:
df3_limpio.groupby(["state", 'city'])["índice_poder_adqui"].mean().max()

In [ ]:
df3_limpio.groupby(["state", 'city'])["price/m2"].mean()

In [ ]:
df3_limpio.groupby(['city'])["price/m2"].max()

In [ ]:
df3_limpio.groupby(['city'])["price/m2"].describe()

In [ ]:
df3_limpio.groupby(["state", 'city'])["price/m2"].min()

In [ ]:
df3_limpio.groupby(["state", 'city'])["price"].min()

In [ ]:
df3_limpio.groupby(["state", 'city'])["price"].mean().round()

In [ ]:
df3_limpio.groupby(["state"])["price"].min()

In [ ]:
df3_limpio.groupby(["state"])["price"].max().round(0)

In [ ]:
df3_limpio.groupby(["state"])["price"].mean().round(0)

In [ ]:
df5 = pd.DataFrame(df3_limpio.groupby(["state", 'city','postal_code'])["price/m2"].mean())
df5

In [ ]:
df4 = pd.DataFrame(df3_limpio.groupby(["state", 'city','postal_code'])["price/m2"].max())
df4

In [ ]:
df4.shape

In [ ]:
df3_limpio.groupby(["state", 'city'])["price/m2"].min()